In [52]:
EVERYONE_TEXT = 'everyone'

import re
import typing
from collections import defaultdict

In [53]:
class Item:
    def __init__(self, name: str, cost: float, people_to_pay: typing.List=[]) -> None:
        self.name = name.strip().lower()
        self.cost = cost
        self.people_to_pay = people_to_pay

    def assign_to_person(self) -> None:
        cost_per_person = self.cost / len(self.people_to_pay)

        for person in self.people_to_pay:
            person.total_due += cost_per_person
            person.items.append(self)

    def __repr__(self) -> str:
        return self.__str__()

    def __str__(self) -> str:
        return f'Item {self.name.capitalize()} with cost {self.cost} and buyers {[person.name.capitalize() for person in self.people_to_pay]}'

class Person:
    def __init__(self, name: str) -> None:
        self.name = name.strip().lower()
        self.items = []
        self.total_due = 0.0

    def get_formatted_items(self) -> typing.List[str]:
        items_with_fractions_formatted = []

        for item in self.items:
            item_with_fraction_formatted = item.name.capitalize()

            if len(item.people_to_pay) != 1:
                item_with_fraction_formatted = f'1/{len(item.people_to_pay)} of {item_with_fraction_formatted}'

            items_with_fractions_formatted.append(item_with_fraction_formatted)
        
        return items_with_fractions_formatted

    def print_detailed_receipt(self) -> None:
        print(f'Name: {self.name}')
        print('Items paid for:')
        
        for item in self.get_formatted_items():
            print(f'\t{item}')
        
        print(f'Grand Total: {self.total_due}')
        print()

    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        return f'{self.name.capitalize()}, in charge of {self.get_formatted_items()} with a total cost of {round(self.total_due, 2)}'

In [54]:
def remove_currency_symbols(cost_with_currency: str) -> str:
    return re.sub(r'[^\d.,]+', '', cost_with_currency)

def get_people_in_line(formatted_people: str, people: typing.List[Person]) -> typing.List[Person]:
    people_in_line = [person.lower() for person in re.findall(r'(\w+)', formatted_people)]

    if 'everyone' in people_in_line:
        return people

    extracted_people = [person for person in people if person.name in people_in_line]

    if len(people_in_line) != len(extracted_people):
        people_not_found = [person.name for person in people if person.name not in people_in_line]
        raise BaseException(f'Some people could not be found in the list of names that you provided: {people_not_found}')

    return extracted_people


def get_line_items(line: str, people: typing.List[Person]) -> typing.List[str]:
    pattern = r'(.+): (.+) \((.+)\)'

    matches = re.findall(pattern, line)
    
    if len(matches) == 0:
        raise BaseException(f'Incorrectly formatted line: {pattern}')

    line_items = matches[0]

    if len(line_items) != 3:
        raise BaseException(f'Incorrectly formatted line: {pattern}')
    
    return [
        line_items[0],
        float(remove_currency_symbols(line_items[1])),
        get_people_in_line(line_items[2], people)
    ]

def get_item_from_line(line: str, people: typing.List[Person]) -> Item:
    item, cost, people_to_pay = get_line_items(line, people)
    return Item(name=item, cost=cost, people_to_pay=people_to_pay)

def get_all_items_from_file(fp: str, people: typing.List[Person]) -> typing.List[Item]:
    with open(fp, 'r') as file:
        lines = file.readlines()
        return [get_item_from_line(line, people) for line in lines]

In [55]:
person_names = input('Please enter all names on this tab, separated by a comma.')

person_names = [name for name in person_names.split(',')]
people = [Person(name=name) for name in person_names]

items = get_all_items_from_file('./tabs/costco/costco-tab-08142022.txt', people)

In [56]:
for item in items:
    item.assign_to_person()

In [57]:
for person in people:
    person.print_detailed_receipt()

Name: jared
Items paid for:
	Lemon juice
	1/3 of Toilet paper
	1/3 of Paper towels
	1/3 of Rinse aid
	1/3 of Dish soap
	1/3 of Bounce dryer sheets
	1/3 of Magic erasers
	1/3 of Steaks
	Gillette razors
	Degree deodorant
	1/3 of Clorox wand
	1/3 of Clorox wipes
	1/3 of Downy scent beads
	1/3 of Bath mat
	1/3 of Tax
Grand Total: 135.41333333333336

Name: carter
Items paid for:
	1/3 of Toilet paper
	1/3 of Paper towels
	Peaches
	1/3 of Rinse aid
	1/3 of Dish soap
	1/3 of Bounce dryer sheets
	1/3 of Magic erasers
	1/3 of Steaks
	Cremo shave cream
	Degree deodorant
	1/3 of Clorox wand
	1/3 of Clorox wipes
	1/3 of Downy scent beads
	1/3 of Bath mat
	1/2 of Bath mat
	1/3 of Tax
Grand Total: 112.90833333333335

Name: taimur
Items paid for:
	1/3 of Toilet paper
	1/3 of Paper towels
	1/3 of Rinse aid
	1/3 of Dish soap
	1/3 of Bounce dryer sheets
	1/3 of Magic erasers
	1/3 of Steaks
	1/3 of Clorox wand
	1/3 of Clorox wipes
	1/3 of Downy scent beads
	1/3 of Bath mat
	1/2 of Bath mat
	1/3 of Tax
Gra